### Testing Evo

#### Imports

In [ ]:
import torch
from IPython.display import display, HTML, Image
from torch_geometric.loader import DataLoader

from settings import *
from generator import generate_organism
from GNCAConv import Conv
from visualizer import Visualizer
from evo_trainer import Evo_Trainer
from enums import EnvironmentType, ModelType
from food_env import *
from online_tracker import OnlineTracker

#%run visualizer.py

#### Train the network

In [ ]:
scale = 4
wall_amount = 100*scale
food_amount = 500*scale
centered_env = Centered_Env(food_amount=food_amount, wall_amount=wall_amount//4)
cluster_env = Cluster_Env(clusters=80*scale, cluster_size=15, wall_amount=wall_amount//3)
circle_env = Circle_Env(circles=3*scale, food_amount=food_amount*4, wall_amount=wall_amount//3)
spiral_env = Spiral_Env(spirals=3, food_amount=food_amount, wall_amount=wall_amount)
bottleneck_env = Bottleneck_Env(food_amount=food_amount//2, wall_amount=(wall_amount*16)//25)
grid_env = Grid_Env(cluster_size=25, grid_size=6*scale)
food_grid_env = Food_Grid_Env(cluster_size=25, grid_size=3*scale)
lab_env = Labyrinth_Env(cluster_size=40, grid_size=(5*scale)//2)

In [ ]:
train_config = Train_Config(stdev=1.0, popsize=10, 
    name='GAT6_localx4', problem_searcher=ProblemSearcher.CMAES,
    with_samplepool=False, timesteps=300)

In [ ]:
generations = 10000

settings = Settings(
    device = torch.device('cpu'),
    batch_size = 1,
    cells = 100,
    #food_envs=[centered_env, cluster_env, circle_env, spiral_env],
    food_envs=[spiral_env, cluster_env, circle_env, grid_env, lab_env, food_grid_env],
    scale = scale,
    wrap_around = False,
    
    model_type = ModelType.Localx4,
    #model_type = ModelType.WithGlobalNode,
    #model_type = ModelType.SmallWorld,

    radius = 0.04,
    radius_food_scale = 5,
    consume_radius_scale = 0.8, 
    noise = 0.002,
    energy_required_to_replicate = 80,
    train_config=train_config,
    max_degree = 8,

    radius_wall_scale = 5,
    radius_wall_damage_scale = 3,
    wall_damage = 30,
    radius_long_scale=4,
)

In [ ]:
#TODO ADD settings like std of clusters..

online_tracker = OnlineTracker(settings)
trainer = Evo_Trainer(settings, online_tracker)
network = Conv(settings=settings)

In [ ]:
generations=10000

In [ ]:
trainer.train(generations)

In [ ]:
network = trainer.get_trained_network()
online_tracker.add_model(network)

#### Visualize trained network

In [ ]:
graphs = [generate_organism(settings).toGraph() for _ in range(settings.batch_size)]
loader = DataLoader(graphs, batch_size=settings.batch_size)
graph = next(iter(loader))
visualizer = Visualizer(settings)
network(graph, 0)

In [ ]:
dpi = 100 #beautiful
#dpi = 50 #fast

anim = visualizer.animate_organism(graph, network) 
visualizer.save_animation_to_gif(anim, train_config.name, fps=10, dpi=dpi)

In [ ]:
Image('../animation/' + train_config.name + '.gif')

In [ ]:
%%javascript
IPython.notebook.save_notebook()

#### Visualization of loss:

In [ ]:
import matplotlib.pyplot as plt
losses = trainer.logger_df
#print(losses)
#plt.plot(losses.stepsize)

if settings.train_config.problem_searcher == ProblemSearcher.GeneticAlgorithm:
    plt.plot(losses.obj0_median_eval)
    plt.plot(losses.obj0_pop_best_eval)
    plt.plot(losses.obj0_mean_eval)
else:
    plt.plot(losses.median_eval)
    plt.plot(losses.pop_best_eval)
    plt.plot(losses.mean_eval)
plt.legend(('median_eval', 'pop_best_eval', 'mean eval'))
plt.show()

In [ ]:
online_tracker.finish()